In [1]:
import random
import pennylane as qml
from pennylane import numpy as np
import sys
sys.path.insert(0,'..')
from maskit.datasets import load_data

In [2]:
# Setting seeds for reproducible results
np.random.seed(1337)
random.seed(1337)

# Loading the data

Data of interest is MNIST data. As we want to go for reproducible results, we
will first go with the option `shuffle=False`. For the rest of the parameters,
we now go with the default options. This gives us data for two classes, the
written numbers 6 and 9. We also only get a limited number of sampes, that is
100 samples for training and 50 for testing. For further details see the
appropriate docstring.

In [3]:
data = load_data("mnist", shuffle=False, target_length=2)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-01-18 23:04:17.587365: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-18 23:04:17.587482: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Setting up a Variational Quantum Circuit for training

There is an example on the [PennyLane website](https://pennylane.ai/qml/demos/tutorial_variational_classifier.html#iris-classification) for iris data showing a setup for a variational classifier. That is variational quantum circuits that can be trained from labelled (classical) data.

In [4]:
wires = 4
layers = 4
epochs = 5
parameters = np.random.uniform(low=-np.pi, high=np.pi, size=(layers, wires, 2))

In [5]:
def variational_circuit(params):
    for layer in range(layers):
        for wire in range(wires):
            qml.RX(params[layer][wire][0], wires=wire)
            qml.RY(params[layer][wire][1], wires=wire)
        for wire in range(0, wires - 1, 2):
            qml.CZ(wires=[wire, wire + 1])
        for wire in range(1, wires - 1, 2):
            qml.CZ(wires=[wire, wire + 1])
    return qml.expval(qml.PauliZ(0))

In [6]:
def variational_training_circuit(params, data):
    qml.templates.embeddings.AngleEmbedding(
        features=data, wires=range(wires), rotation="X"
    )
    return variational_circuit(params)

In [7]:
dev = qml.device('default.qubit', wires=wires, shots=1000)
circuit = qml.QNode(func=variational_circuit, device=dev)
training_circuit = qml.QNode(func=variational_training_circuit, device=dev)

In [8]:
circuit(parameters)

tensor(-0.052, requires_grad=True)

In [9]:
training_circuit(parameters, data.train_data[0])

tensor(0.014, requires_grad=True)

In [10]:
print(training_circuit.draw())

AttributeError: 'QNode' object has no attribute 'draw'

In [14]:
# some helpers
def correctly_classified(params, data, target):
    prediction = training_circuit(params, data)
    if prediction < 0 and target[0] > 0:
        return True
    elif prediction > 0 and target[1] > 0:
        return True
    return False

def overall_cost_and_correct(cost_fn, params, data, targets):
    cost = correct_count = 0
    for datum, target in zip(data, targets):
        cost += cost_fn(params, datum, target)
        correct_count += int(correctly_classified(params, datum, target))
    return cost, correct_count

In [15]:
# Playing with different cost functions
def crossentropy_cost(params, data, target):
    prediction = training_circuit(params, data)
    scaled_prediction = prediction + 1 / 2
    predictions = np.array([1 - scaled_prediction, scaled_prediction])
    return cross_entropy(predictions, target)

def distributed_cost(params, data, target):
    """Cost function distributes probabilities to both classes."""
    prediction = training_circuit(params, data)
    scaled_prediction = prediction + 1 / 2
    predictions = np.array([1 - scaled_prediction, scaled_prediction])
    return np.sum(np.abs(target - predictions))

def cost(params, data, target):
    """Cost function penalizes choosing wrong class."""
    prediction = training_circuit(params, data)
    predictions = np.array([0, prediction]) if prediction > 0 else np.array([prediction * -1, 0])
    return np.sum(np.abs(target - predictions))

In [16]:
optimizer = qml.AdamOptimizer()
cost_fn = cost

In [17]:
start_cost, correct_count = overall_cost_and_correct(cost_fn, parameters, data.test_data, data.test_target)
print(f"start cost: {start_cost}, with {correct_count}/{len(data.test_target)} correct samples")

start cost: 56.230000000000004, with 24/50 correct samples


In [23]:
params = parameters.copy()
for _ in range(epochs):
    for datum, target in zip(data.train_data, data.train_target):
        params = optimizer.step(lambda weights: cost_fn(weights, datum, target), params)

    cost, correct_count = overall_cost_and_correct(cost_fn, params, data.test_data, data.test_target)
    print(f"epoch{_} cost: {cost}, with {correct_count}/{len(data.test_target)} correct samples")

epoch0 cost: 43.85400000000002, with 29/50 correct samples
epoch1 cost: 38.38000000000001, with 34/50 correct samples
epoch2 cost: 33.187999999999995, with 38/50 correct samples
epoch3 cost: 31.218000000000004, with 39/50 correct samples
epoch4 cost: 31.368, with 39/50 correct samples


In [24]:
final_cost, correct_count = overall_cost_and_correct(cost_fn, params, data.test_data, data.test_target)
print(f"final cost: {final_cost}, with {correct_count}/{len(data.test_target)} correct samples")

final cost: 31.08, with 40/50 correct samples
